In [208]:
# Code in file autograd/two_layer_net_custom_function.py
import torch
import torch.nn.functional as F  # useful stateless functions
import time

class MyReLU(torch.autograd.Function):
  """
  We can implement our own custom autograd Functions by subclassing
  torch.autograd.Function and implementing the forward and backward passes
  which operate on Tensors.
  """
  @staticmethod
  def forward(ctx, x):
    """
    In the forward pass we receive a context object and a Tensor containing the
    input; we must return a Tensor containing the output, and we can use the
    context object to cache objects for use in the backward pass.
    """
    ctx.save_for_backward(x)
    return x.clamp(min=0)

  @staticmethod
  def backward(ctx, grad_output):
    """
    In the backward pass we receive the context object and a Tensor containing
    the gradient of the loss with respect to the output produced during the
    forward pass. We can retrieve cached data from the context object, and must
    compute and return the gradient of the loss with respect to the input to the
    forward function.
    """
    x, = ctx.saved_tensors
    grad_x = grad_output.clone()
    grad_x[x < 0] = 0
    return grad_x


class TanLU(torch.autograd.Function):
  @staticmethod
  def forward(ctx, x):
    ctx.save_for_backward(x)
    return torch.max(x, x.tanh())

  @staticmethod
  def backward(ctx, grad_output):
    x, = ctx.saved_tensors
    derivative = (1.0 - x.tanh().pow(2))
    derivative[x >= 0] = 1
    return derivative * grad_output


class SinLU(torch.autograd.Function):
  @staticmethod
  def forward(ctx, x):
    ctx.save_for_backward(x)
    return torch.max(x, x.sin())

  @staticmethod
  def backward(ctx, grad_output):
    x, = ctx.saved_tensors
    return torch.max(x.abs() / x, x.cos()) * grad_output
    
"""@staticmethod
  def backward(ctx, grad_output):
    x, = ctx.saved_tensors
    derivative = x.cos()
    derivative[x >= 0] = 1
    return derivative * grad_output"""

#device = torch.device('cpu')
device = torch.device('cuda') # Uncomment this to run on GPU

# N is batch size; D_in is input dimension;
# H is hidden dimension; D_out is output dimension.
N, D_in, H, D_out = 64, 1000, 100, 10

# Create random Tensors to hold input and output
x = torch.randn(N, D_in, device=device)
y = torch.randn(N, D_out, device=device)

# Create random Tensors for weights.
w1 = torch.randn(D_in, H, device=device, requires_grad=True)
w2 = torch.randn(H, D_out, device=device, requires_grad=True)

tic = time.time()
learning_rate = 1e-6
for t in range(15001):
  # Forward pass: compute predicted y using operations on Tensors; we call our
  # custom ReLU implementation using the MyReLU.apply function
  #y_pred = MyReLU.apply(x.mm(w1)).mm(w2)
  #y_pred = TanLU.apply(x.mm(w1)).mm(w2)
  y_pred = SinLU.apply(x.mm(w1)).mm(w2)
  #y_pred = F.relu(x.mm(w1)).mm(w2)

 
  # Compute and print loss
  loss = (y_pred - y).pow(2).sum()
  if t % 500 == 0:
    print(t, loss.item())

  # Use autograd to compute the backward pass.
  loss.backward()

  with torch.no_grad():
    # Update weights using gradient descent
    w1 -= learning_rate * w1.grad
    w2 -= learning_rate * w2.grad

    # Manually zero the gradients after running the backward pass
    w1.grad.zero_()
    w2.grad.zero_()
    
print("%f seconds" % (time.time() - tic))



0 31594060.0
500 1.3092007066006772e-05
1000 1.9149656509398483e-06
1500 8.756911711316206e-07
2000 5.159846523383749e-07
2500 3.619117308062414e-07
3000 2.992068459661823e-07
3500 2.3256262693394092e-07
4000 2.0740695561016764e-07
4500 1.7284612852108694e-07
5000 1.540995810955792e-07
5500 1.4288161764852703e-07
6000 1.2391731729621824e-07
6500 1.120927421993656e-07
7000 1.132237557044391e-07
7500 9.936493938766944e-08
8000 8.687638342053106e-08
8500 8.070866641674002e-08
9000 8.007982899016497e-08
9500 8.192574796339613e-08
10000 7.20720692015675e-08
10500 6.80708325262458e-08
11000 7.241099808652507e-08
11500 6.348947323431275e-08
12000 6.48314681939155e-08
12500 6.171907784846553e-08
13000 6.255173445879336e-08
13500 6.563429622019612e-08
14000 5.877193132164393e-08
14500 5.917036816072141e-08
15000 6.654447304299538e-08
10.645319 seconds


In [205]:
import cupy as cp
import time
# N is batch size; D_in is input dimension;
# H is hidden dimension; D_out is output dimension.
N, D_in, H, D_out = 64, 1000, 100, 10

# Create random input and output data
x = cp.random.randn(N, D_in)
y = cp.random.randn(N, D_out)

# Randomly initialize weights
w1 = cp.random.randn(D_in, H)
w2 = cp.random.randn(H, D_out)


activation_function = "relu"
#activation_function = "sinlu"

tic = time.time()
learning_rate = 1e-6
for t in range(15001):
  if activation_function == "relu":
    # Forward pass: compute predicted y
    h = x.dot(w1)
    h_relu = cp.maximum(h, 0)
    y_pred = h_relu.dot(w2)
  
  elif activation_function == "sinlu":
    # Forward pass: compute predicted y
    h = x.dot(w1)
    h_relu = cp.maximum(h, cp.sin(h))
    y_pred = h_relu.dot(w2)
  
  # Compute and print loss
  loss = cp.square(y_pred - y).sum()
  if t % 50 == 0:
    print(t, loss)
  
  if activation_function == "relu":
    # Backprop to compute gradients of w1 and w2 with respect to loss
    grad_y_pred = 2.0 * (y_pred - y)
    grad_w2 = h_relu.T.dot(grad_y_pred)
    grad_h_relu = grad_y_pred.dot(w2.T)
    grad_h = grad_h_relu.copy()
    grad_h[h < 0] = 0
    grad_w1 = x.T.dot(grad_h)
    
  elif activation_function == "sinlu":
    # Backprop to compute gradients of w1 and w2 with respect to loss
    grad_y_pred = 2.0 * (y_pred - y)
    grad_w2 = h_relu.T.dot(grad_y_pred)
    grad_h_relu = grad_y_pred.dot(w2.T)
    grad_h = cp.maximum(cp.abs(h_relu) / h_relu, cp.cos(h_relu)) * grad_h_relu
    grad_w1 = x.T.dot(grad_h)
  
 
  # Update weights
  w1 -= learning_rate * grad_w1
  w2 -= learning_rate * grad_w2
    
print("%f seconds" % (time.time() - tic))


0 28780567.84166371
50 9181.32251046368
100 239.82248553894152
150 10.389517949663194
200 0.5277910813482501
250 0.029034486204176358
300 0.0016846171303719604
350 0.00010178335299635097
400 6.353529894686354e-06
450 4.0754736680296224e-07
500 2.6766009550100157e-08
550 1.7953261118913527e-09
600 1.227776182430677e-10
650 8.55065118004578e-12
700 6.058354706981365e-13
750 4.36399086264833e-14
800 3.193750653299734e-15
850 2.3728210792700794e-16
900 1.7885525044032605e-17
950 1.3662050020153237e-18
1000 1.05868281838963e-19
1050 9.106638679547816e-21
1100 1.345539471399972e-21
1150 3.8919537038164444e-22
1200 1.7152115219569862e-22
1250 9.607182952921477e-23
1300 6.410461448269717e-23
1350 4.741612016306246e-23
1400 3.638092200533159e-23
1450 2.956010319609922e-23
1500 2.5171239965591414e-23
1550 2.110716842759161e-23
1600 1.8177211414913563e-23
1650 1.5976113840993354e-23
1700 1.410310557458918e-23
1750 1.2742243473899044e-23
1800 1.1582171912293511e-23
1850 1.0817493293200936e-23
1900